In [11]:
import cv2
import imutils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

import gdown as gdown
import zipfile

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets

In [12]:
PATH = os.getcwd()
letterToNumber = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
numberToLetter = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [38]:
# Descarga los modelos entrenados
'''url =  'https://drive.google.com/uc?id=18wmT6d4-5tAIuCeS4KkAzCg7yuSARy5m&export=download'
output = 'Models/models.zip'
gdown.download(url, output, quiet=False)
with zipfile.ZipFile(PATH + "/Models/models.zip","r") as zip_ref:
    zip_ref.extractall(PATH + "/Models/")'''

Downloading...
From: https://drive.google.com/uc?id=1PGKJ4GU9yI1JVVddAtvuyhyPfL719BUG&export=download
To: /Users/alessiayi/Documents/Github/Utec/ai_project5/Models/models.zip
100%|██████████| 222M/222M [00:07<00:00, 30.5MB/s] 


In [13]:
from Models.Models import *

In [22]:
#Obtener images = lista de np arrays que entran al modelo
def preprocessing(image_name):
    img = cv2.imread(PATH + "\\" + image_name)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
    mask = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel)
    conts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    conts = imutils.grab_contours(conts)
    boxes = []
    for cntr in conts:
        x,y,w,h = cv2.boundingRect(cntr)
        cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 1)
        boxes.append((x,y,w,h))
    #plt.imshow(img)
    #plt.show()
    spaces = []

    if (len(boxes) == 0):
        return boxes, spaces
    
    boxes = sorted(boxes, key=lambda x: (x))
    # print(len(boxes))
    prom = 0
    spaceLetters = []
    for i in range(len(boxes)-1):
        x = boxes[i][0]
        y = boxes[i][1]
        w = boxes[i][2]
        h = boxes[i][3]
        tmp = boxes[i+1][0]-(x+w)
        prom+=tmp
        spaceLetters.append(tmp)
    if (len(boxes) > 1):
        prom = prom/(len(boxes)-1) + 5
    else:
        prom = 0
    # print("prom: ", math.ceil(prom))
    for i in range(len(spaceLetters)):
        # print(boxes[i+1][0]-(x+w), math.ceil(prom))
        if (spaceLetters[i]) > math.ceil(prom):
            spaces.append(i)

    # print(spaces)
    # cont = 0
    images = []
    for e, i in enumerate(boxes):
        x = i[0]
        y = i[1]
        w = i[2]
        h = i[3]
        cropped = thresh[y:y+h, x:x+w]
        # inverted = 255 - cropped
        # print(cropped.shape)
        height = cropped.shape[0]
        width = cropped.shape[1]
        if (height > width):
            diff = height - width
            padding = cv2.copyMakeBorder(cropped,30,30,30+int(diff/2),30+int(diff/2),cv2.BORDER_CONSTANT,value=[0,0,0])
        elif (height == width):
            padding = cv2.copyMakeBorder(cropped,30,30,30,30,cv2.BORDER_CONSTANT,value=[0,0,0])
        else:
            diff = width - height
            padding = cv2.copyMakeBorder(cropped,30+int(diff/2),30+int(diff/2),30,30,cv2.BORDER_CONSTANT,value=[0,0,0])
        resized = cv2.resize(padding, (28, 28))
        for ii in range(resized.shape[0]):
            for jj in range(resized.shape[1]):
                if resized[ii][jj] > 0:
                    resized[ii][jj] = 255
        images.append(resized.astype('float32'))    
        # if e < 9:
        #plt.imshow(resized)
        #plt.show()
        # if cont in spaces:
        #     blankSpace = np.zeros([28,28,1],dtype=np.uint8)
        #     blankSpace.fill(255)
        #     images.append(blankSpace.astype('float32'))  
        #     plt.imshow(blankSpace)
        #     plt.show()
        # cont+=1
    return images, spaces
    print(len(images))
    print(type(images[0]))
    print(images[0].shape)

In [15]:
#Testea
def tester(images, spaces, model):
    transformToTensor = transforms.ToTensor()
    imageTensors = [transformToTensor(np.array(img)) for img in images]
    imageBatch = torch.utils.data.DataLoader(dataset=imageTensors, batch_size=64, shuffle=False)

    batch = 0
    predictText = []
    for imgBatch in imageBatch:
        imgBatch = imgBatch.to(device)
        labelBatchPred = model(imgBatch)

        labelBatchPred = labelBatchPred.to('cpu')
        imgBatch = imgBatch.to('cpu')
        #print(labelBatchPred[0])
        _, predBatch = torch.max(labelBatchPred, 1)

        group = 0
        n = len(imgBatch)
        ok = 1
        while(group < 4 and ok):
            plt.figure(figsize = (25, 10))
            r, c = 1, 16
            for i in range(16):
                plt.subplot(r, c, i+1)
                plt.imshow(imgBatch[16*group + i].permute(1, 2, 0))
                #plt.xlabel(str(64*batch + 16*group + i + 1) + ": " + numberToLetter[predBatch[16*group + i].item()] + " (" + numberToLetter[np.argmax(y_data[16*group + i])] + ")", fontsize = 15)
                predictLetter = numberToLetter[predBatch[16*group + i].item()]
                predictText.append(predictLetter)
                plt.xlabel(str(64*batch + 16*group + i + 1) + ": " + predictLetter,fontsize = 15)
                if 16*group + i + 1 == n:
                    ok = 0
                    break
            plt.show()
            #print("")
            group += 1
        batch += 1

    finalText = ""
    for i in range(len(predictText)):
        finalText += predictText[i]
        # print(predictText[i])
        if i in spaces:
            finalText += " "
            # print(" ")
    return finalText

In [16]:
# Carga el modelo y lo envia a device
#model = torch.load(PATH + '/Models/AlexNet.pt', map_location=torch.device('cpu'))
model1 = torch.load(PATH + '/Models/Lenet5.pt')
model1 = model1.to(device)
model1.eval()

model2 = torch.load(PATH + '/Models/AlexNet.pt')
model2 = model2.to(device)
model2.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (11): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=26, bias=True

In [47]:
import cv2
import time

def initiate_camera(seconds, out_name):
    st = time.time()
    iteration = 0

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    while True:
        ret, frame = cap.read()
        # frame = cv2.resize(frame, None, fx = 0.5, fy = 0.5, interpolation=cv2.INTER_AREA)
        cv2.imshow('Input', frame)
        cur_time = time.time()
        elapsed_time = cur_time - st

        if elapsed_time > iteration * seconds:
            cv2.imwrite(out_name, frame)
            iteration += 1
            #Aquí debería ir el código que manda la imagen (frame) al separador de letras.
            images, spaces = preprocessing(out_name)
            text1 = ""
            text2 = ""
            if len(images) > 0:
                text1 = tester(images, spaces, model1)
                text2 = tester(images, spaces, model2)
            else:
                text1 = ""
                text2 = ""
            #frame contiene la imagen como valor, y la imagen capturada como archivo se guarda en out_name
            print(text1)
            print(text2)
        c = cv2.waitKey(1)
        if c == 27:
            break
    cap.release()
    cv2.destroyAllWindows()


In [49]:
initiate_camera(10, "out.jpg")

[ WARN:0@15906.367] global /tmp/opencv-20220430-60527-1xmsjni/opencv-4.5.5/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/Users/alessiayi/Documents/Github/Utec/ai_project5\out.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.5.5) /tmp/opencv-20220430-60527-1xmsjni/opencv-4.5.5/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 